In [1]:
import os
with open("../env.sh", 'r') as f:
    for line in f.read().split("\n"):
        key, value = line[len('export '):].split('=', maxsplit=1)
        os.environ[key] = value
        print(f"{key}={value}")

HAM_RADIO_JSON=/net/vol/jenkins/jsons/ham_radio.json
HAM_RADIO_JSON_PATH=/net/vol/jenkins/jsons/ham_radio.json
HAM_RADIO_DB=/net/db/ham_radio/
MODEL_DIR=/net/vol/werning/ham_radio/models/
LIBRISPEECH_JSON_PATH=/net/vol/jenkins/jsons/librispeech.json
LIBRISPEECH_VAD_PATH=/net/vol/werning/jsons/librispeech_vad.json
LIBRISPEECH_VAD_JSON_PATH=/net/vol/werning/jsons/librispeech_with_vad.json
RIR_SCENARIO_JSON_PATH=/net/db/rirs_sms_wsj_8spk_16k/scenarios.json
MUSAN_JSON_PATH=/net/vol/jenkins/jsons/musan.json
MEETING_VAD_JSON_PATH=/net/vol/werning/jsons/vad_meetings.json


In [6]:
# from ham_radio.database.ham_radio.create_vad_json import create
# create()

Processing VAD for dataset: dev_clean
Processing VAD for dataset: dev_other
  Missing VAD for 2864 of 2864 examples in dataset dev_other
Processing VAD for dataset: test_clean
Processing VAD for dataset: test_other
  Missing VAD for 2939 of 2939 examples in dataset test_other
Processing VAD for dataset: train_clean_100
Processing VAD for dataset: train_clean_360
Processing VAD for dataset: train_other_500
  Missing VAD for 148688 of 148688 examples in dataset train_other_500


In [3]:
from ham_radio.database.ham_radio import LibrispeechMeeting
librispeech = LibrispeechMeeting()

In [4]:
from tqdm import trange
# create datasets
# ham_radio: train: 122h, dev 19h, eval 37h
def create_set(name, num_samples):
    print("name", name, "num_samples", num_samples)
    ds = librispeech.get_raw_dataset(name)
    ds_raw = []
    it = iter(ds)
    for _ in trange(num_samples):
        ds_raw.append(next(it))
    ds_raw_map = {ex['example_id']: ex for ex in ds_raw}
    return ds_raw_map
    
num_samples_train = 122 * 60 // 4 # 4 min per audio
train_raw = create_set('train', num_samples_train) 

num_samples_dev = 19 * 60 // 4 # 4 min per audio
dev_raw = create_set('dev', num_samples_dev)

num_samples_test = 37 * 60 // 4 # 4 min per audio
test_raw = create_set('test', num_samples_test)

name train num_samples 1830


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1830/1830 [00:10<00:00, 171.37it/s]


name dev num_samples 285


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 285/285 [00:02<00:00, 140.07it/s]


name test num_samples 555


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 555/555 [00:03<00:00, 157.01it/s]


In [5]:
import paderbox as pb
pb.io.dump({'datasets': {'train':train_raw, 'dev':dev_raw, 'test':test_raw}}, "/net/vol/werning/jsons/vad_meetings.json")